# Create a Local Ethernet (Layer 2) Network

This notebook shows how to create an isolated local Ethernet and connect compute nodes to it.  


## Import the FABlib Library


In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/work/fabric_config/tokens.json
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
Bastion Username,pruth_0031379841
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  This example uses components of model `NIC_Basic` which are SR-IOV Virtual Function on a 100 Gpbs Mellanox ConnectX-6 PCI device. The VF is accessed by the node via PCI passthrough. Other NIC models are listed below. When using dedicated PCI devices the whole physical device is allocated to one node and the device is accessed by the node using PCI passthrough. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port.  Either port can be connected to the network.

Next, add an `l2network` to the slice and pass the list of interfaces you want connected to this Ethernet. If all interfaces in the list are located on the same site, the network will automatically be a local Ethernet.  By default, a node is put on a random site.  If you want to ensure that your nodes are all on the same site you can specify the name of the site in the `add_node` methode.  You can use the `fablib.get_random_site()` method to get a random site name that can be used for both nodes.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [2]:
slice_name = 'MySlice2'
site = fablib.get_random_site()
print(f"Site: {site}")

node1_name = 'Node1'
node2_name = 'Node2'

network_name='net1'

Site: NCSA


In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

# Node1
node1 = slice.add_node(name=node1_name, site=site).enable_docker()
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)

# Node2
node2 = slice.add_node(name=node2_name, site=site).enable_docker()
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)


#Submit Slice Request
slice.submit();


Retry: 9, Time: 469 sec


ID,8915b3c3-2615-40a1-a964-2aba170fc225
Name,MySlice2
Lease Expiration (UTC),2023-02-27 07:37:13 +0000
Lease Start (UTC),2023-02-26 07:37:14 +0000
Project ID,990d8a8b-7e50-4d13-a3be-0f133ffa8653
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
116da349-5fc2-4fab-8265-c73b91711998,Node1,2,8,10,default_rocky_8,qcow2,ncsa-w1.fabric-testbed.net,NCSA,rocky,2620:0:c80:1001:f816:3eff:fe23:a48f,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:0:c80:1001:f816:3eff:fe23:a48f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7192655b-c6b8-4b7f-9804-2e90a375ed51,Node2,2,8,10,default_rocky_8,qcow2,ncsa-w1.fabric-testbed.net,NCSA,rocky,2620:0:c80:1001:f816:3eff:fedb:d0dc,Active,,ssh -t -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:0:c80:1001:f816:3eff:fedb:d0dc,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
044f0e86-356e-4ec0-82b1-592f931d2fe8,net1,L2,L2Bridge,NCSA,192.168.1.0/24,None,Active,


Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address
Node1-nic1-p1,Node1,net1,100,auto,,16:94:86:41:50:8E,eth1,eth1,192.168.1.1
Node2-nic1-p1,Node2,net1,100,auto,,16:33:C6:8B:4F:05,eth1,eth1,192.168.1.2



Time to print interfaces 476 seconds


In [4]:
for node in slice.get_nodes():
    #if 'router' not in node.get_name():
    print(node.get_name())
    attributes = node.upload_directory('/home/fabric/work/docker_containers','.')
    node.execute("cd docker_containers/fabric_multitool ; docker compose up -d", quiet=True, output_file=f"{node.get_name()}.log");
    node.execute("docker exec fabric_multitool sudo ./tools/host_tune_redhat.sh", quiet=True, output_file=f"{node.get_name()}.log");
        

Node1
Node2


In [ ]:
import json
slice = fablib.get_slice(slice_name)

node = slice.get_node(name=node1_name)     

# for nono-host networks, remvoe:  "bridge": None 
daemon_json = {
    "registry-mirrors": ["https://registry.ipv4.docker.com"],
    "iptables": True,
    "ip6tables": True,
    "experimental": True,
    "ip-forward": True,
    "ip-masq": True,
    "ipv6": True,
    "fixed-cidr-v6": "fd8d:73ee:3857:7fab::/64",
    "bridge": None,
}


daemon_json_str = json.dumps(daemon_json, indent=4).replace('"','\\"')

print(daemon_json_str)


stdout, stderr = node.execute(f"sudo sh -c \"echo '{daemon_json_str}' > /etc/docker/daemon.json \" " )

#. sudo sh -c "echo 'I'\''m a programmer' > /path/to/file"



## Run the Experiment

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()